In [4]:
import numpy as np
import pandas as pd

class BRKGA:

    
    #Genetic Parameters
    _son1 = []
    _son2 = []
    _capacity = []
    _cost =[]
    _m = 0
    _obj = 0
    #Population Parameters
    _psize = 0
    _csize = 0
    _elite = 0
    _mutant = 0
    #Number of Iterations
    _n = 0
    
    def __init__(self, cp_array, ct_array, c_size, p_size, elite, mutant, n_it, obj = False):
        
        """
        cp_array -> list of the capacities of the facilities sorted by index order
        
        ct_array -> list of the costs of the facilities sorted by index order
        
        c_size -> int vale, size of the chromosome
        
        p_size -> int value, size of the population
        
        m -> float value, represents the limit to simultaneous facilities
        
        elite -> int value, number of elites per population
        
        mutant -> int value, number of mutants per population
        
        n_it -> int value, number of iterations
        
        obj -> bool value, False(to return value) or True (to return cover)
        """
        if(type(cp_array) is list or type(cp_array) is np.ndarray):
            self._cover = cp_array
        else:
            raise Exception("The value of the parameter \"cp_array\" is invalid, please enter a valid list or ndarray.")
        
        if(type(ct_array) is list or type(ct_array) is np.ndarray):
            self._cost = ct_array
        else:
            raise Exception("The value of the parameter \"ct_array\" is invalid, please enter a valid list or ndarray.")

        if(type(p_size) is int):
            self._psize = p_size
        else:
            raise Exception("The value of the parameter \"p_size\" is invalid, please enter a valid integer.")
            
        if(type(c_size) is int):
            self._csize = c_size
        else:
            raise Exception("The value of the parameter \"c_size\" is invalid, please enter a valid integer.")
            
        if(type(elite) is int):
            self._elite = elite
        else:
            raise Exception("The value of the parameter \"elite\" is invalid, please enter a valid integer.")
            
        if(type(mutant) is int):
            self._mutant = mutant
        else:
            raise Exception("The value of the parameter \"mutant\" is invalid, please enter a valid integer.")
            
        
        if(type(n_it) is int):
            self._n = n_it
        else:
            raise Exception("The value of the parameter \"n_it\" is invalid, please enter a valid integer value.")
        
        if(type(obj) is bool):
            self._obj = obj
        else:
            raise Exception("The value of the parameter \"obj\", please enter a valid boolean.")
         

    def Encoder(self, size):
        gene = pd.Series(np.random.rand(size))
        return gene
    
    def Decoder(self,encoded_list,obj):
        
        decoded_list = encoded_list.sort_values(kind='mergesort').index
        value = 0
        cover = 0
        gene = []
        for i in decoded_list[0:self._csize]:
            #cost of the current element
            value+=self._cost[i]
            cover += self._cover[i]
            gene.append(i)
                
        if obj == False:
            return value, gene
        else:
            return cover, gene
        
    def Crossover(self,parent1, parent2):
        
        son = list(parent1)
        p1 = pd.Series(parent1)
        p2 = pd.Series(parent2)

        tpl = []

        common_values = []
        for i in p1.index:
            if p1[i] in p2.values:
                common_values.append(p1[i])

        tpl = {x:0 for x in common_values}
        for i in range(0,len(common_values)):
            tpl[common_values[i]] =  (p1[p1==common_values[i]].index[0],p2[p2 == common_values[i]].index[0])


        for i in range(0,len(p1)):
            coin = np.random.rand()
            if(coin <= 0.5):
                if p1[i] in common_values:
                    son[tpl[p1[i]][1]],son[tpl[p1[i]][0]] =son[tpl[p1[i]][0]],son[tpl[p1[i]][1]]
                elif p2[i] in common_values:
                    son[tpl[p2[i]][1]],son[tpl[p2[i]][0]] =son[tpl[p2[i]][0]],son[tpl[p2[i]][1]]
                else:
                    son[i] = p2.loc[i]
        return son
                
    def Elite(self,solutions, population, e):
        #creates the array of elite chromossomes
        elite = []
        #creates the array of the "e" best solutions
        elite_sol = solutions.sort_values(ascending=False, kind='mergesort')
        #iterates for e
        for i in range(0,e):
            #appends to the elite array the best solutions and removes
            #it from the population to avoid duplicity issues
            elite.append(population.pop(elite_sol.index[i]))
        return elite_sol, elite
    
    def Mutant(self,m):
        mutant = []
        mutant_sol = []
        for i in range(0,m):
            encoded_gene = self.Encoder(len(self._cost))
            sol, decoded_gene = self.Decoder(encoded_gene,self._obj)
            mutant.append(decoded_gene)
            mutant_sol.append(sol)
        return mutant_sol, mutant
    
    
    def Solve(self):
        
        count = 0
        next_population = []
        next_solutions = pd.Series()
        best_solutions = []
        best_chromossomes = []
        while(count < self._n):
            population = []
            solutions = pd.Series()
            elite = []
            elite_sol= []
            mutant = []
            mutant_sol = []
            #First Population
            if count == 0:
                
                #creates the population
                for i in range(0,self._psize):
                    chromossome = self.Encoder(len(self._cost))
                    obj, decoded_chromossome = self.Decoder(chromossome,self._obj)
                    solutions = solutions.append(pd.Series(obj),ignore_index=True)
                    population.append(decoded_chromossome)
                #creates the elites chromossomes
                elite_sol, elite = self.Elite(solutions,population,self._elite)
                #creates the mutant chromossomes
                mutant_sol, mutant = self.Mutant(self._mutant)
                
                #adds the elite chromossomes to the next generation
                for i in range(0,self._elite):
                    next_population.append(elite[i])
                    next_solutions = next_solutions.append(pd.Series(elite_sol[i]),ignore_index=True)
                    if not(elite[i] in best_chromossomes):
                        best_chromossomes.append(elite[i])
                        best_solutions.append(elite_sol[i])
                    
                #adds the mutant chromossomes to the next generation
                for i in range(0,self._mutant):
                    next_population.append(mutant[i])
                    next_solutions = next_solutions.append(pd.Series(mutant_sol[i]),ignore_index=True)

                #calculates how many chromossomes are needed to complete the population
                p_left = self._psize - self._elite - self._mutant
                
                for i in range(0,p_left):
                    elite_parent = elite[np.random.randint(0,self._elite)]
                    
                    normal_parent = population[np.random.randint(0,len(population))]
                    son = self.Crossover(elite_parent,normal_parent)
                    
                    
                    #cost of the son's solution to cost-wise objective
                    cost = 0
                    #cover of the son's solution to cover-wise objective
                    cover = 0
                    
                    for j in son:
                        if self._obj == False:
                            cost += self._cost[j]
                        else:
                            cover+= self._cover[j]
                    
                    sol = 0
                    
                    if self._obj == False:
                        sol = cost
                    else:
                        sol = cover
                    next_population.append(son)
                    next_solutions = next_solutions.append(pd.Series(sol),ignore_index=True)
            else:
                population = []
                solutions = pd.Series()
                
                #appends the previous value of next_population to the popultation array
                for i in next_population:
                    population.append(i)
                #and clears the next_population to 
                next_population = []
                
                for i in next_solutions.index:
                    solutions = solutions.append(pd.Series(next_solutions.loc[i]),ignore_index=True)
                    
                next_solutions = pd.Series()
                next_population = []
                
                #creates the elites chromossomes
                elite_sol, elite = self.Elite(solutions,population,self._elite)
                #creates the mutant chromossomes
                mutant_sol, mutant = self.Mutant(self._mutant)
            
                 #adds the elite chromossomes to the next generation
                for i in range(0,self._elite):
                    next_population.append(elite[i])
                    next_solutions = next_solutions.append(pd.Series(elite_sol[i]),ignore_index=True)
                    if not(elite[i] in best_chromossomes):
                        best_chromossomes.append(elite[i])
                        best_solutions.append(elite_sol[i])
                    
                #adds the mutant chromossomes to the next generation
                for i in range(0,self._mutant):
                    next_population.append(mutant[i])
                    next_solutions = next_solutions.append(pd.Series(mutant_sol[i]),ignore_index=True)
                    
                #calculates how many chromossomes are needed to complete the population
                p_left = self._psize - self._elite - self._mutant
                
                for i in range(0,p_left):
                    elite_parent = elite[np.random.randint(0,self._elite)]
                    normal_parent = population[np.random.randint(0,len(population))]
                    son = self.Crossover(elite_parent,normal_parent)
                    
                    
                    #cost of the son's solution to cost-wise objective
                    cost = 0
                    #cover of the son's solution to cover-wise objective
                    cover = 0
                    
                    for j in son:
                        if self._obj == False:
                            cost += self._cost[j]
                        else:
                            cover+= self._cover[j]
                    
                    sol = 0
                    
                    if self._obj == False:
                        sol = cost
                    else:
                        sol = cover
                    next_population.append(son)
                    next_solutions = next_solutions.append(pd.Series(sol),ignore_index=True)
            count+=1
        return best_solutions, best_chromossomes